In [1]:
!pip install faker


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 20.7 MB/s eta 0:00:00


In [2]:
import random
import pandas as pd
from faker import Faker
from datetime import datetime, timedelta

fake = Faker()

departments = ['Apparel', 'Electronics', 'Kitchenware', 'Fitness', 'Stationery', 'Toys', 'Books', 'Beauty']
items = []
for i in range(1, 17):
    item = {
        'item_id': f'I{i}',
        'item_name': fake.word().capitalize(),
        'price': random.randint(5, 1000),
        'department': random.choice(departments)
    }
    items.append(item)
items_df = pd.DataFrame(items)

customers = []
for i in range(1, 51):
    customers.append({
        'customer_id': f'C{i}',
        'first_name': 'John' if i % 10 == 0 else fake.first_name(),
        'last_name': 'Doe' if i % 10 == 0 else fake.last_name(),
        'Address': fake.address().replace("\n", ", ")
    })
customers_df = pd.DataFrame(customers)

sales = []
dates_2022_2023 = [datetime(2022, 1, 1) + timedelta(days=random.randint(0, 820)) for _ in range(50)]
for i in range(1, 51):
    item = random.choice(items)
    quantity = random.randint(1, 5)
    sale = {
        'Date': dates_2022_2023[i-1].strftime('%Y-%m-%d'),
        'Order_id': f'O{i}',
        'Item_id': item['item_id'],
        'Customer_id': random.choice(customers)['customer_id'],
        'Quantity': quantity,
        'Revenue': item['price'] * quantity
    }
    sales.append(sale)
sales_df = pd.DataFrame(sales)

print("Customers")
print(customers_df.head())

print("\nItems")
print(items_df.head())

print("\nSales")
print(sales_df.head())


Customers
  customer_id first_name last_name  \
0          C1   Lawrence      Mann   
1          C2       Mary      Diaz   
2          C3       Gina     Adams   
3          C4     Sophia   Johnson   
4          C5     Brandy     Ayers   

                                             Address  
0  21938 Thomas Mission Apt. 509, Michaeltown, DC...  
1  12460 Gordon Point Suite 668, New Alicia, AS 9...  
2                   Unit 0460 Box 1615, DPO AE 08222  
3  467 Christina Fords Apt. 642, South Paulabury,...  
4              03293 Brown Wall, New Becky, IL 23939  

Items
  item_id item_name  price   department
0      I1  Business    342  Electronics
1      I2  Democrat    528  Electronics
2      I3    Family    962      Apparel
3      I4    Common    318      Fitness
4      I5     Mouth    131      Apparel

Sales
         Date Order_id Item_id Customer_id  Quantity  Revenue
0  2022-11-01       O1     I12         C19         5      115
1  2022-03-23       O2      I5         C18         1 

In [7]:
print (sales_df.)

          Date Order_id Item_id Customer_id  Quantity  Revenue
0   2022-11-01       O1     I12         C19         5      115
1   2022-03-23       O2      I5         C18         1      131
2   2024-03-08       O3      I6         C31         3     1617
3   2022-09-24       O4      I3          C6         5     4810
4   2022-12-30       O5      I9         C27         2      862
5   2023-10-01       O6     I16         C10         2      948
6   2023-08-15       O7      I5         C48         3      393
7   2024-03-02       O8      I3          C5         1      962
8   2023-10-25       O9      I4         C43         2      636
9   2023-04-05      O10     I16         C35         5     2370
10  2022-05-13      O11     I13          C9         1      655
11  2022-05-20      O12      I3          C9         5     4810
12  2023-10-06      O13      I2         C14         3     1584
13  2023-02-06      O14     I12         C16         1       23
14  2023-10-09      O15      I5          C8         4  

In [5]:
import sqlite3

conn = sqlite3.connect(":memory:")

customers_df.to_sql("CUSTOMERS", conn, index=False, if_exists='replace')
items_df.to_sql("ITEMS", conn, index=False, if_exists='replace')
sales_df.to_sql("SALES", conn, index=False, if_exists='replace')


50

In [8]:
query = """
SELECT COUNT(*) AS total_orders
FROM SALES
WHERE Date = '2023-03-18'
"""

pd.read_sql_query(query, conn)


,total_orders
0,1


In [9]:
query = """
SELECT COUNT(*) AS john_doe_orders
FROM SALES S
JOIN CUSTOMERS C ON S.Customer_id = C.customer_id
WHERE S.Date = '2023-03-18'
  AND C.first_name = 'John'
  AND C.last_name = 'Doe'
"""

pd.read_sql_query(query, conn)


,john_doe_orders
0,0


In [10]:
query = """
SELECT COUNT(DISTINCT Customer_id) AS total_customers,
       AVG(customer_total) AS avg_spent_per_customer
FROM (
    SELECT Customer_id, SUM(Revenue) AS customer_total
    FROM SALES
    WHERE Date BETWEEN '2023-01-01' AND '2023-01-31'
    GROUP BY Customer_id
)

"""

pd.read_sql_query(query, conn)


,total_customers,avg_spent_per_customer
0,0,None


In [11]:
query = """
SELECT I.department, SUM(S.Revenue) AS total_revenue
FROM SALES S
JOIN ITEMS I ON S.Item_id = I.item_id
WHERE S.Date BETWEEN '2022-01-01' AND '2022-12-31'
GROUP BY I.department
HAVING SUM(S.Revenue) < 600



"""

pd.read_sql_query(query, conn)

,department,total_revenue
0,Toys,539


In [13]:
query ="""SELECT MAX(Revenue) AS max_order_revenue,
       MIN(Revenue) AS min_order_revenue
FROM SALES
"""

pd.read_sql_query(query, conn)



,max_order_revenue,min_order_revenue
0,4830,23
